In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def bollinger_band_backtest(
    ticker='SPY',
    start_date='2018-01-01',
    end_date='2023-01-01',
    period=20,
    num_std=2
):
    """
    A robust Bollinger Band trading strategy with checks to avoid alignment or empty-data errors.
    """

    # 1. Fetch data from yfinance with debug output
    print(f"Fetching data for {ticker}...")
    df = yf.download(ticker, start=start_date, end=end_date)

    # Print the columns we received
    print(f"Columns in downloaded data: {df.columns.tolist()}")
    print(f"First few rows of data:\n{df.head()}")

    # 2. Check if data was returned
    if df.empty:
        print(f"No data returned from yfinance for {ticker} in {start_date} to {end_date}.")
        return

    # 3. Handle MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        # Select the 'Close' price column for the specified ticker
        close_prices = df['Close'][ticker]
    else:
        # If not MultiIndex, assume single-level columns
        close_prices = df['Close']

    # Create a new DataFrame with just the close prices
    df = pd.DataFrame({'Close': close_prices})

    # 4. Calculate Bollinger Bands
    df['MA'] = df['Close'].rolling(window=period).mean()
    df['STD'] = df['Close'].rolling(window=period).std()
    df['Upper'] = df['MA'] + (num_std * df['STD'])
    df['Lower'] = df['MA'] - (num_std * df['STD'])

    # 5. Drop any rows that are NaN after the rolling calculations
    print(f"Number of rows before dropping NaN: {len(df)}")
    df.dropna(subset=['Close', 'MA', 'STD', 'Upper', 'Lower'], inplace=True)
    print(f"Number of rows after dropping NaN: {len(df)}")

    if df.empty:
        print("All rows became NaN after Bollinger Band calculations. Possibly the date range is too short.")
        return

    # 6. Generate signals
    df['Signal'] = 0
    df.loc[df['Close'] < df['Lower'], 'Signal'] = 1
    df.loc[df['Close'] > df['Upper'], 'Signal'] = -1

    # Forward-fill signals
    df['Position'] = df['Signal'].replace(to_replace=0, method='ffill')

    # 7. Compute strategy returns
    df['Market_Return'] = df['Close'].pct_change()
    df['Strategy_Return'] = df['Position'].shift(1) * df['Market_Return']
    df.dropna(subset=['Strategy_Return'], inplace=True)

    if df.empty:
        print("No data left after shifting positions. Strategy cannot be computed.")
        return

    # 8. Calculate cumulative returns
    df['Cumulative_Strategy'] = (1 + df['Strategy_Return']).cumprod()
    df['Cumulative_BuyHold']  = (1 + df['Market_Return']).cumprod()

    # 9. Print performance metrics
    strategy_return = df['Cumulative_Strategy'].iloc[-1] - 1
    buyhold_return  = df['Cumulative_BuyHold'].iloc[-1] - 1

    print("\nPerformance Summary:")
    print(f"Final Strategy Return: {strategy_return:.2%}")
    print(f"Final Buy & Hold Return: {buyhold_return:.2%}")

    # 10. Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Cumulative_Strategy'], label='Bollinger Strategy', color='blue')
    plt.plot(df.index, df['Cumulative_BuyHold'], label='Buy & Hold', color='orange')
    plt.title(f"Bollinger Band Strategy vs. Buy & Hold ({ticker})")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")
    plt.legend()
    plt.show()

    # Plot Bollinger Bands
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Close'], label='Close Price', color='black')
    plt.plot(df.index, df['Upper'], label='Upper Band', color='red', linestyle='--')
    plt.plot(df.index, df['MA'], label='Moving Average', color='blue', linestyle='--')
    plt.plot(df.index, df['Lower'], label='Lower Band', color='green', linestyle='--')
    plt.title(f"Bollinger Bands ({ticker})")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

    return df

# Example usage:
if __name__ == "__main__":
    df_result = bollinger_band_backtest(
        ticker='SPY',
        start_date='2018-01-01',
        end_date='2023-01-01',
        period=20,
        num_std=2
    )